# Checking density

In [ ]:
#custom function
from get_neighbor_counts import *
from neighbor_search_within import *
from neighbor_search_across import *
from ecdf import ecdf
#rgeneral packages
import tifffile as tf
import matplotlib.pyplot as plt
#enhance figure display
%config InlineBackend.figure_format = 'retina'

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_2d_locs_on_2d_image(df_locs_2d_1, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d_1.x,
        y=df_locs_2d_1.y,
        mode='markers',
        marker_symbol='circle',
        marker=dict(
            #maxdisplayed=1000,
            size=5, color = "green"
            ),
        name = "Decoded"
        )
    )
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='circle',
            marker=dict(
                #maxdisplayed=1000,
                size=5, color = "red"
                ),
            name = "Undecoded"
            )
        )
    #-------------------------------------------
    
    fig.show()
    
def plot_3d_locs_on_2d_image(df_tiff_1, df_tiff_2, tiff, channel, 
                             raw_src = None, raw_image = False, add_trace = True, zmax=10):
    
    if raw_image == False:

        #PLot All Z's that had dot detection
        #-------------------------------------------
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1, None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)
    else:
        #read raw image
        tiff = tf.imread(raw_src)
        if len(tiff.shape) == 3:
            tiff = tiff.reshape(1,tiff.shape[0],tiff.shape[1],tiff.shape[2])
        print("shape =", tiff.shape)
        #plot
        for z in range(len(tiff[:,channel])):
            if add_trace == False:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,None, tiff[z, channel], zmax=zmax, add_trace=add_trace)
            else:
                df_locs_2d_1 = df_tiff_1[(df_tiff_1.z > z-1) & (df_tiff_1.z < z+1)]
                df_locs_2d_2= df_tiff_2[(df_tiff_2.z > z-1) & (df_tiff_2.z < z+1)]
                plot_2d_locs_on_2d_image(df_locs_2d_1,df_locs_2d_2, tiff[z, channel],add_trace=add_trace, zmax=zmax)

# Density Visualization

In [ ]:
def keep_dots_in_cells(mask, dot_locations):
    """
    A function to remove any dots outside of mask
    Parameter
    ---------
    mask = cellpose generated mask path
    dot_locations = dot_locations path
    """
    
    #read in data
    locations = pd.read_csv(dot_locations)
    #cellpose mask outputs (c,y,x)
    img = tf.imread(mask)
    #get x and y coordinates
    locations_xy = locations[["x","y"]].values.astype(int)
    dot_info = []
    #keep dots only in cells
    for i in range(len(locations)):
        x = locations_xy[i][0]
        y = locations_xy[i][1]
        if img[y,x] == 0:
            continue
        else:
            cell = img[y,x]
            dot_info.append([i,cell])
            
    dot_info = np.array(dot_info)
    
    #keep rows that have cells
    dots_in_cells = locations.loc[dot_info[:,0]]
    
    #add cell info
    dots_in_cells["cell number"] = dot_info[:,1]
    
    return dots_in_cells

In [ ]:
#read in files
dots_used_locations = f""
dots_unused_locations = f""

In [ ]:
#map dots
mask = f""
dots_used = keep_dots_in_cells(mask, dots_used_locations)
dots_unused = keep_dots_in_cells(mask, dots_unused_locations)

In [ ]:
#check all rounds (could also isolate single round)
dots_used_round = dots_used[dots_used["hyb"]<18]
dots_unused_round = dots_unused[dots_unused["hyb"]<18]

In [ ]:
#read in image
img = tf.imread(f"")

In [ ]:
#combine
comb = pd.concat([dots_used_round, dots_unused_round])
#sort
comb = comb.sort_values("hyb").reset_index(drop=True)

In [ ]:
#take a look
comb

In [ ]:
#check each seed (0,1,2,3) or all
counts, index = neighbor_counts(comb, hybs=18, num_barcodes=3, seed="all", radius=1)
comb["neighbors"] = counts

In [ ]:
def image_density(df_locs_2d, df_locs_2d_2, img_2d,add_trace = True, zmax=1000):
    
    #For Plotting 2d image
    #-------------------------------------------
    fig = px.imshow(
        img_2d,
        width=700,
        height=700,
        binary_string=True,
        binary_compression_level=4,
        binary_backend='pil',
        zmax = zmax
    )
    #-------------------------------------------
    
    #For Plotting 2d dots
    #-------------------------------------------
    fig.add_trace(go.Scattergl(
        x=df_locs_2d.x,
        y=df_locs_2d.y,
        mode='markers',
        marker_symbol='circle',
        marker=dict(
            #maxdisplayed=1000,
            size=5, color = comb["neighbors"],  
            colorscale="Viridis", showscale=True
            ),
        name = "Collapsed Dots"
        ))
    
    if add_trace == True:
        fig.add_trace(go.Scattergl(
            x=df_locs_2d_2.x,
            y=df_locs_2d_2.y,
            mode='markers',
            marker_symbol='circle',
            marker=dict(
                #maxdisplayed=1000,
                size=5, color="red"
                ),
            name = "fakes"
            ))
        
        fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01))
    
        
    fig.show()
    
    

In [ ]:
#read in gene locations file if you want to overlay on top of used dots
genes = pd.read_csv(f"")

In [ ]:
#isolate fakes
fakes = genes[genes["genes"].str.startswith("fake")]

In [ ]:
#take a look
fakes

In [ ]:
#collapsed pseudocolors
image_density(comb, df_locs_2d_2=None,  img_2d=img[0][2],add_trace = False, zmax=1000)

In [ ]:
#collapsed pseudocolors and overlay fakes
image_density(comb,fakes, img[0][2],add_trace = True, zmax=1000)

In [ ]:
#separate locations by barcoding round
barcoding_round = []
hybs=18
locations=comb
num_barcodes=3
hyb_rounds = np.arange(0,hybs,1)
temp = []
for h in hyb_rounds:
    if h == hyb_rounds[len(hyb_rounds)-1]:
        barcode = locations[locations["hyb"] == h]
        temp.append(barcode)
        comp_round = pd.concat(temp)
        barcoding_round.append(comp_round) 
    elif (h % (hybs/num_barcodes) != 0) or (h == 0):
        barcode = locations[locations["hyb"] == h]
        temp.append(barcode)
    else:
        comp_round = pd.concat(temp)
        barcoding_round.append(comp_round)
        temp = []
        barcode = locations[locations["hyb"] == h]
        temp.append(barcode)

In [ ]:
#take a look
barcoding_round[0]

In [ ]:
#plot each round and color code
fig = px.imshow(
    img[0][2],
    width=700,
    height=700,
    binary_string=True,
    binary_compression_level=4,
    binary_backend='pil',
    zmax = 1000
)
i=0
color = ["blue","red","green","orange"]
round_label = ["Round 1", "Round 2", "Round 3", "Round 4"]
for df in barcoding_round:
    fig.add_trace(go.Scattergl(
        x=df.x,
        y=df.y,
        mode='markers',
        marker_symbol='circle',
        marker=dict(
            #maxdisplayed=1000,
            size=5, color = color[i]
            ),
        name = round_label[i]
        )
    )
    i += 1


fig.show()

# Density calculations

In [ ]:
channel=4
dots= f""
mask = f""
dots = keep_dots_in_cells(mask, dots)

In [ ]:
dots=dots[["hyb","ch","x","y","z","size","peak intensity","average intensity"]]

In [ ]:
#calculate number of neighbors with expanding search radius
seeds= [0,1,2]
radii = np.linspace(0.5,3,10)

neighbor_list = []
for seed in seeds:
    for radius in radii:
        neighbors = neighbor_search(dots, hybs=45, num_barcodes=3, seed=seed, radius=radius)
        mean_neighbor = count_neighbors(neighbors)
        neighbor_list.append([seed,mean_neighbor,radius])

In [ ]:
#convert to df
neighbor_df = pd.DataFrame(neighbor_list)

In [ ]:
neighbor_df

In [ ]:
#plot the average number of neighbors across rounds using different seeds with various radii
for seed in neighbor_df[0].unique():
    plt.plot(neighbor_df[neighbor_df[0]==seed][2],neighbor_df[neighbor_df[0]==seed][1], 
             label = f"Seed {seed}", linewidth=0.5)
plt.legend()
plt.xlabel("Radius in Pixels")
plt.ylabel("Average number of neighbors")
plt.show()

In [ ]:
#perform various radius searches and calculate dot density
neigh_list = []
index_list = []
radius_list = [0.5,1,1.5,2]
for radius in radius_list:
    neighbors,seed = neighbor_search_within(dots, hybs=45, num_barcodes=3, seed=0, radius=radius)
    density,index = density_estimate(neighbors, radius=radius, average=False)
    index_list.append(index)
    neigh_list.append(density)

In [ ]:
#combine dfs
comb = pd.concat(neigh_list).reset_index(drop=True)

In [ ]:
#plot distributions
ecdf.plot(comb, label_column = "radius", val_column = "number of neighbors/pixel", 
          conf = False, color = ["red","blue","green","purple","orange"])
plt.xlabel("# of neighbors/pixel")
plt.ylabel("ECDF")
plt.xlim(-0.05,2.5)
plt.show()

In [ ]:
#plot distributions
ecdf.plot(comb, label_column = "radius", val_column = "number of neighbors/pixel", 
          conf = False, color = ["red","blue","green","purple","orange"])
plt.xlabel("# of neighbors/pixel")
plt.ylabel("ECDF")
plt.show()

In [ ]:
dots= pd.read_csv(f"")

In [ ]:
cell_mask = tf.imread(mask)

In [ ]:
len(dots[dots["hyb"]<15])/(cell_mask>0).sum()